<a name="top" id="top"></a>

<div align="center">
    <h1>Graver Augmentation Multiseed Algorithm</h1>
    <a href="https://github.com/bernalde">David E. Bernal Neira</a>
    <br>
    <i>Davidson School of Chemical Engineering, Purdue University</i>
    <br>
    <i>Universities Space Research Association</i>
    <br>
    <i>NASA QuAIL</i>
    <br>
    <br>
    <a href="https://github.com/pedromxavier">Pedro Maciel Xavier</a>
    <br>
    <i>Computer Science &amp; Systems Engineering Program, Federal University of Rio de Janeiro</i>
    <br>
    <i>PSR Energy Consulting &amp; Analytics</i>
    <br>
    <br>
    <a href="https://colab.research.google.com/github/pedromxavier/QUBO-notebooks/blob/main/notebooks/3-GAMA.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
    <a href="#installation">
        <img src="https://img.shields.io/badge/⚙️-Installation_Instructions-blue" alt="Installation Instructions"/>
    </a>
    <a href="https://bernalde.github.io/">
        <img src="https://img.shields.io/badge/⚗️-Bernal_Lab-blue" alt="Bernal Lab"/>
    </a>
</div>

### Activate Environment

In [ ]:
import Pkg; Pkg.activate(@__DIR__) # Here we go!

## About this notebook

This notebook makes simple computations of Graver basis.
Because of the complexity of these computation, we suggest that for more complicated problems you install the excellent **[4ti2](https://4ti2.github.io/)** software, an open-source implementation of several routines useful for the study of integer programming through algebraic geometry.
It can be used as a stand-alone library and call it from C++ or directly from Julia.
In Julia, a binding is provided by **[lib4ti2_jll](https://github.com/JuliaBinaryWrappers/lib4ti2_jll.jl)**.

## Introduction to GAMA

The Graver Augmentation Multiseed Algorithm (GAMA) was proposed by [two](https://arxiv.org/abs/1902.04215) [papers](https://arxiv.org/abs/1907.10930) by Alghassi, Dridi, and Tayur from the CMU Quantum Computing group.
The three main ingredients of this algorithm, designed to solve integer programs with linear constraints and nonlinear objective, are:

- Computing the Graver basis (or a subset of it) of an integer program.
- Performing an augmentation.
- Given that only for certain objective functions, the Graver augmentation is guaranteed to find a globally optimal solution, the algorithm is initialized in several points.

This algorithm can be adapted to take advantage of Quantum Computers by leveraging them as black-box Ising/QUBO problem solvers.
In particular, obtaining several feasible solution points for the augmentation and computing the Kernel of the constraint matrix can be posed as QUBO problems.
After obtaining these solutions, other routines implemented in classical computers are used to solve the optimization problems, making this a hybrid quantum-classical algorithm.

### Introduction to Graver basis computation

A Graver basis is defined as

$$
\mathcal{G}(\mathbf{A}) = \bigcup_{j} \mathcal{H}_{j}(\mathbf{A})
$$

where $\mathcal{H}_{j}(\mathbf{A})$ are the minimal Hilbert basis of $\mathbf{A}$ in each orthant.

Equivalently we can define the Graver basis as the $\sqsubseteq$-minimal set of a lattice

$$
\mathcal{L}(\mathbf{A}) = \left\lbrace{}{\mathbf{x} : \mathbf{A} \mathbf{x} = 0, \mathbf{x} \in \mathbb{Z}^{n}}\right\rbrace{} \setminus \left\lbrace{}{0}\right\rbrace{}
$$

Here we won't interact with the Quantum Computer.
However, we will obtain the Graver basis of a problem using package 4ti2.
This notebook studies the behavior of the search algorithm in the case that we only have available a subset of the Graver basis.

## Problem statement

We will be solving EXAMPLE 4 in the code, which corresponds to Case 2 in the original GAMA paper.
The problem is derived from finance and deals with the maximization of expected returns on investments and the minimization of the variance.

$$
\begin{array}{rll}
    \displaystyle \min & \displaystyle -\sum_{i = 1}^{n} \mu_{i} x_{i} + \sqrt{\frac{1 - \varepsilon}{\varepsilon} \sum_{i = 1}^{n} \sigma_{i}^2 x_{i}^2 } \\
    \textrm{s.t.}      & A \mathbf{x} = \mathbf{b} \\
    ~                  & \mathbf{x} \in \left\lbrace{}{-2, -1, 0, 1, 2}\right\rbrace{}^{n}
\end{array}
$$

### Example

Let

$$
A = \begin{bmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 \\
1 & 1 & 1 & 1 & 0 & 1 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 1 & 1 & 1 \\
0 & 1 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 1 & 0 & 1 & 1 & 0 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 1 & 0 \\
\end{bmatrix}
$$

This particular instance of convex INLP has $m = 5$, $n = 25$, $\varepsilon = 0.01$, $\mu_{i} = \textrm{rand}[0, 1]$, $\sigma_{i} = \textrm{rand}[0, \mu_{i}]$.
$A \in \mathbb{B}^{m \times n}$ and each $b_{j}$ is half the sum of the $j$-th row of $A$.
In this example, $\mathbf{b} = \left({9, 8, 7, 5, 5}\right)'$.

First we would write this problem as a an unconstrained one by penalizing the linear constraints as quadratics in the objective.
Let's first define the problem parameters.

In [5]:
A = [
    1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0
    1 1 1 1 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 1
    0 1 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 1 1 1
    0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 0 0 0
    0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0
]

m, n = size(A)

b = ceil.(Int, sum(A; dims = 2) / 2)

5×1 Matrix{Int64}:
 9
 8
 7
 5
 5

In [6]:
x0 = [1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -2, 1, 0, -1, 0, 1, -1, 1, -2, -2, 1, 1, 1]

xl = fill(-2, n)
xu = fill(2, n);

In [11]:
ε = 0.01
μ = rand(n)      # ~ [0, 1]
σ = rand(n) .* μ # ~ [0, μ]

25-element Vector{Float64}:
 0.005557116751233621
 0.17653683032560488
 0.006393592028502544
 0.8611321959286385
 0.26543188380689614
 0.07001148481489934
 0.4935925888076086
 0.16746461972487978
 0.4256054954439059
 0.0700846103504365
 ⋮
 0.5279803514158558
 0.06949578764730337
 0.07434998696622722
 0.23868494434363138
 0.5054747201950235
 0.20099362608668975
 0.31717194219795364
 0.39041844978408696
 0.02378749193752114

In [2]:
using lib4ti2_jll, BinaryWrappers
const lib4ti2_bin = @generate_wrappers(lib4ti2_jll);

In [4]:
function write_mat(path::AbstractString, A)
    m, n = size(A)

    open(path, "w") do io
        println(io, "$m $n")

        join(io, (join(@view(A[i, :]), " ") for i = 1:m), "\n")
    end

    return nothing
end

function read_mat(path::AbstractString, type = Int)
    m, n = parse.(Int, split(readline(path)))

    A = Matrix{type}(undef, m, n)

    open(path, "r") do io
        readline(io)

        for (i, line) in enumerate(eachline(io))
            A[i, :] .= parse.(Int, split(line))
        end
    end

    return A
end

function graver_basis(A, xl, xu)
    G = nothing

    mktempdir() do path
        proj_path = joinpath(path, "proj")

        write_mat("$(proj_path).mat", A)
        write_mat("$(proj_path).lb", xl)
        write_mat("$(proj_path).ub", xu)

        run(`$(lib4ti2_bin)/graver -q $(proj_path)`)

        run(`cat "$(proj_path).gra"`)

        G = read_mat("$(proj_path).gra")
    end

    return G
end

graver_basis (generic function with 1 method)

In [7]:
G = graver_basis(A, xl', xu')

26292×25 Matrix{Int64}:
 0   0   0   0   0   0  1  0  0   0   0  …   0  0  0  -1  0   0  0  0   0  0
 1   0   0   0   0   0  0  0  0   0   0      0  0  0  -1  0  -1  0  0   0  0
 0   0   0   0   0   0  0  0  1   0   0      0  0  0  -1  0   0  0  0   0  0
 0   0   0   0   0   0  0  0  0   0   0      0  0  1  -1  0  -1  0  0   0  0
 0   1  -1   0   0   0  0  0  0   0   0      0  0  0   0  0   0  0  0   0  0
 0   1   0  -1   0   0  0  0  0   0   0  …   0  0  0   0  0   0  0  0   0  0
 0   1   0   0  -1   0  0  0  0   0   0      0  0  0   0  0  -1  0  0   0  0
 0   1   0   0   0  -1  0  0  0   0   0      0  0  0   0  0   0  0  0   0  0
 0   1   0   0   0   0  0  1  0  -1   0      0  0  0  -1  0  -2  0  0   0  0
 0   0   0   0   0   0  0  1  0   0  -1      0  0  0  -1  0   0  0  0   0  0
 ⋮                   ⋮                ⋮  ⋱   ⋮                ⋮            
 2  -2   0   0   0   0  0  0  0   2   0     -2  0  0   0  0   0  0  0  -1  2
 2  -2   0   0   0  -1  0  0  0   2   0     -2  0  0 

In [8]:
graver_basis([1 1 1 1; 1 5 10 25], [0 0 0 0], [15 15 15 15])

0×4 Matrix{Int64}

# References

- [Julia Colab Notebook Template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb)
- [QuIPML22](https://github.com/bernalde/QuIPML22/blob/main/notebooks/Notebook%201%20-%20LP%20and%20IP.ipynb)

<a name="installation" id="installation"></a>

# Installation

## Colab Instructions

If not in a Colab notebook, continue to the next section.

1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other required packages. This can take a couple of minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell

bash <(curl -s "https://raw.githubusercontent.com/pedromxavier/QUBO-notebooks/main/scripts/install-colab-julia.sh")

## Validate Installation

In [ ]:
versioninfo()

## Local Installation Instructions

If you don't have a Julia installation yet, consider using [juliaup](https://github.com/JuliaLang/juliaup).
Otherwise, run the next cell to install the necessary packages.

### Install Julia Packages

In [ ]:
import Pkg

Pkg.activate(@__DIR__)
Pkg.instantiate(;io = devnull) # Suppress Output

<div align="center">
    <a href="#top">🔝 Go back to the top 🔝</a>
</div>